In [ ]:
import json, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['axes.facecolor'] = 'white'
import matplotlib.colors

import pytz 

from datetime import timedelta, datetime

from aquabyte.data_access_utils import RDSAccessUtils

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
penIdData = {}

In [ ]:
daySummariesData = {}

In [ ]:
penIds = [95, 83, 84, 1, 5, 119, 66, 37, 56, 60, 61]

In [ ]:
columns = ['quality_score']

start_date = '2020-05-01'
end_date = '2020-06-04'
    
for penId in penIds:
    print(penId)
    
    if penId in penIdData:
        continue
    
    pen_id = penId
    

    rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
    query = """
            select captured_at, left_crop_metadata 
            from prod.crop_annotation
            where pen_id={0}
            and group_id='{0}'
            and captured_at between '{1}' and '{2}'
            and annotation_state_id in (1, 8)
            and service_id=1;
        """.format(pen_id, start_date, end_date)
    print(query)
    df = rds_access_utils.extract_from_database(query)
    df = df.sort_values('captured_at', ascending=True)
    df.index = pd.to_datetime(df.captured_at)
    df['hour'] = df.index.hour.values
    df['date'] = df.index.date.astype(str)
    vals = df.left_crop_metadata.apply(lambda x: x.get('quality_score') if x else None)
    df['quality_score'] = vals
    
    myFilter = (df['hour'] >= 6) & (df['hour'] <= 17)
    myFilter2 = (df['hour'] < 6) | (df['hour'] > 17)
    
    myData = {}
    
    myData['hourlyAvg'] = df[columns].rolling('1H').mean().resample('H').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    myData['dailyAvg'] = df[columns].rolling('1D').mean().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    myData['hourlyAvg1'] = df[myFilter][columns].rolling('1H').mean().resample('H').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    myData['dailyAvg1'] = df[myFilter][columns].rolling('1D').mean().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    myData['hourlyAvg2'] = df[myFilter2][columns].rolling('1H').mean().resample('H').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    myData['dailyAvg2'] = df[myFilter2][columns].rolling('1D').mean().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)

    myData['byHour'] = df.groupby(['hour']).mean()
    myData['byHour1'] = df[myFilter].groupby(['hour']).mean()
    myData['byHour2'] = df[myFilter2].groupby(['hour']).mean()
    
    penIdData[penId] = myData


In [ ]:
startDate = '2020-05-01'
endDate = '2020-06-04'
    
for penId in penIds:
    daySummariesQuery = """
        select date, num_lati_fish
        from day_summaries a
        where a.pen_id = %i
        and a.date >= '%s'
        and a.date < '%s';
    """ % (penId, startDate, endDate)

    if penId in daySummariesData:
        continue
        
    print(daySummariesQuery)

    rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
    daySummaries = rds_access_utils.extract_from_database(daySummariesQuery)
        
    daySummaries.index = daySummaries['date']
    daySummaries = daySummaries.sort_index()

    daySummariesData[penId] = daySummaries.copy()
        

In [ ]:
query = """
    select p.id as id, s.name as site_name, p.name as pen_name from customer.pens p
    left join customer.sites s
    on p.site_id = s.id
    order by p.id;
"""

df_pens = rds_access_utils.extract_from_database(query)

In [ ]:
fig, axes = plt.subplots(nrows = len(penIds), ncols = 1, figsize = (10, 10))
fig.set_size_inches(20, len(penIds) * 10)
fig.patch.set_facecolor('#E0E0E0')

for index, penId in enumerate(penIds):
    print(penId)
    foundPenData = df_pens[df_pens.id == penId].values
    
    if len(foundPenData) > 0:
        foundPen = foundPenData[0]
        siteName = foundPen[1]
        penName = foundPen[2]
    else:
        siteName = 'N/A'
        penName = 'N/A'
    
    myData = penIdData[penId]
    daySummaries = daySummariesData[penId]
    
    axes[index].bar(myData['dailyAvg'].index, np.log(myData['dailyAvg']['quality_score']))
    #axes[index].bar(myData['dailyAvg2'].index, np.log(myData['dailyAvg2']['quality_score']), alpha = 0.5)
    axes[index].axhline(y = np.log(0.01), linestyle = '--')
    axes[index].set_title('%s %s (%i) Image Score Analysis' % (siteName, penName, penId))
    axes[index].set_xlabel('Date')
    axes[index].set_ylabel('Logarithm of image score')
    axes[index].set_ylim((-20, 0))
    
    twinAxis = axes[index].twinx()
    twinAxis.set_ylim((0, 70))
    
    twinAxis.bar(daySummaries.index, daySummaries['num_lati_fish'], color = 'red', alpha = 0.25)
    twinAxis.set_ylabel('QA-ed image KPI')

In [ ]:
fig, axes = plt.subplots(nrows = 6, ncols = 1, figsize = (10, 10))
fig.set_size_inches(20, 40)

filter0 = (hourlyAvg1.index > '2020-05-16') & (hourlyAvg1.index < '2020-05-19')
filter1 = (hourlyAvg1.index > '2020-05-11') & (hourlyAvg1.index < '2020-05-15')
filter2 = (hourlyAvg1.index > '2020-05-21') & (hourlyAvg1.index < '2020-05-27')
filter3 = (hourlyAvg1.index > '2020-06-01') & (hourlyAvg1.index < '2020-06-04')
filter02 = (hourlyAvg2.index > '2020-05-16') & (hourlyAvg2.index < '2020-05-19')
filter12 = (hourlyAvg2.index > '2020-05-11') & (hourlyAvg2.index < '2020-05-15')
filter22 = (hourlyAvg2.index > '2020-05-21') & (hourlyAvg2.index < '2020-05-27')
filter32 = (hourlyAvg2.index > '2020-06-01') & (hourlyAvg2.index < '2020-06-04')

axes[0].bar(dailyAvg.index, np.log(dailyAvg['quality_score']))
axes[0].bar(dailyAvg2.index, np.log(dailyAvg2['quality_score']), alpha = 0.5)
axes[0].set_title('Vikane Pen 1 Image Score Analysis')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Logarithm of image score')
axes[2].bar(hourlyAvg[filter0].index, np.log(hourlyAvg[filter0]['quality_score']), width = 0.02)
axes[2].bar(hourlyAvg2[filter02].index, np.log(hourlyAvg2[filter02]['quality_score']), width = 0.02)
axes[2].set_title('On 5/18 around 8:28, camera is repositioned, improves images')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Logarithm of image score')
axes[1].bar(hourlyAvg[filter1].index, np.log(hourlyAvg[filter1]['quality_score']), width = 0.02)
axes[1].bar(hourlyAvg2[filter12].index, np.log(hourlyAvg2[filter12]['quality_score']), width = 0.02)
axes[1].set_title('On 5/12 around 9:50, camera is repositioned, worsens images')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Logarithm of image score')
axes[3].bar(hourlyAvg[filter2].index, np.log(hourlyAvg[filter2]['quality_score']), width = 0.02)
axes[3].bar(hourlyAvg2[filter22].index, np.log(hourlyAvg2[filter22]['quality_score']), width = 0.02)
axes[3].set_title('On 5/22, lights go off, come back on 5/25')
axes[3].set_xlabel('Date')
axes[3].set_ylabel('Logarithm of image score')
axes[4].bar(hourlyAvg[filter3].index, np.log(hourlyAvg[filter3]['quality_score']), width = 0.02)
axes[4].bar(hourlyAvg2[filter32].index, np.log(hourlyAvg2[filter32]['quality_score']), width = 0.02)
axes[4].set_title('On 6/2, new camera installed in new position')
axes[4].set_xlabel('Date')
axes[4].set_ylabel('Logarithm of image score')
axes[5].bar(byHour.index, byHour['quality_score'])
axes[5].bar(byHour2.index, byHour2['quality_score'])
axes[5].set_title('Vikane Pen 1 Image Score by Hour of Day')
axes[5].set_xlabel('Hour of Day')
axes[5].set_ylabel('Image score')

In [ ]:
vikane = dailyAvg